## Pre-Processing DataBase Loader

A idéia desse notebook é criar uma base de dados no formato Torch para os arquivos em pdf, junto com uma pré-classificação de treino/teste como pré-processamento do `LayoutLMv3`. 

- A primeira atividade é converter os arquivos pdfs em imagens `.jpg`, similar à organização de um dataset de cats vs dogs (competição clássica do Kaggle para Redes Neurais Convolucionais)
- A segunda atividade é classificar os relatórios enquanto um diretório acima por empresas de auditoria (**Third Party**)
- A terceira atividade é usar o `OpenCV` para delimitar os `blinding boxs` nas áreas de cada página de arquivo `.jpg` , declarar seu respectivo label e enriquecer com variáveis, para cada relatório em subdiretórios classificados por empresa de auditoria (**Third Party**) 

### passo 1: converter pdfs para imagens

In [2]:
from pathlib import Path
from pdf2image import convert_from_path
from tqdm import tqdm

In [ ]:
import os
from pdf2image import convert_from_path
from tqdm import tqdm

def convert_pdfs_to_images(pdf_directory, output_directory, dpi=120):
    """
    Converte um conjunto de PDFs em imagens, armazenando as páginas de cada PDF em subdiretórios separados.

    Args:
        pdf_directory (str): Caminho para o diretório contendo os arquivos PDF.
        output_directory (str): Caminho para o diretório onde os subdiretórios e imagens serão armazenados.
        dpi (int): Resolução das imagens geradas (padrão: 150 DPI).
    """
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)

    pdf_files = [f for f in os.listdir(pdf_directory) if f.lower().endswith('.pdf')]

    for pdf_file in tqdm(pdf_files, desc="Convertendo PDFs", unit="arquivo"):
        pdf_path = os.path.join(pdf_directory, pdf_file)

        pdf_name = os.path.splitext(pdf_file)[0]
        pdf_output_dir = os.path.join(output_directory, pdf_name)
        os.makedirs(pdf_output_dir, exist_ok=True)

        try:
            images = convert_from_path(pdf_path, dpi=dpi)

            for page_number, image in enumerate(tqdm(images, desc=f"Processando {pdf_file}", unit="página", leave=False), start=1):
                image_filename = f"page_{page_number}.jpg"
                image_path = os.path.join(pdf_output_dir, image_filename)
                image.save(image_path, 'JPEG')

        except Exception as e:
            print(f"Erro ao processar o arquivo '{pdf_file}': {e}")

In [81]:
ROOT = Path.cwd().parent.parent
RAW_PDF = ROOT / 'data' / 'external_data' / 'audit_reports'
PROCESSED_PDF = ROOT / 'data' / 'processed_data' / 'audit_reports_database'

In [83]:
print(PROCESSED_PDF.parent)

/home/cleyton/Documentos/ProjetosGit/meu_repositorio_dissertacao/data/processed_data


In [ ]:
convert_pdfs_to_images(pdf_directory=RAW_PDF, output_directory=PROCESSED_PDF)

Convertendo PDFs: 100%|██████████| 244/244 [03:34<00:00,  1.14arquivo/s]


### passo 2: classificar os relatórios em um diretório acima de acordo com as empresas de auditoria

In [5]:
import shutil
import re
from PIL import Image
import pytesseract

In [85]:
# Regex para identificar as empresas 
EMPRESA_REGEX = {
    "Food Chain ID": r"\b[Ff]ood\s*[Cc]hain\s*[Ii][Dd]\b",
    "Control Union": r"\b[Cc]ontrol\s*[Uu]nion\b",
    "Genesis Certificações": r"\b[Gg]enesis\s\b",
    "Bureau Veritas": r"\b[Bb]ureau\s*[Vv]eritas\b",
    "SGS Argentina": r"\b[SGS]{3}\s*[Aa]rgentina\b",
    "Cert ID": r"\b[Cc]ert\s*[Ii][Dd]\b",
}


ORGANIZED_PDF = PROCESSED_PDF.parent / 'organized_audit_reports'

In [ ]:
os.makedirs(ORGANIZED_PDF, exist_ok=True)

def identificar_empresa(texto):
    for empresa, regex in EMPRESA_REGEX.items():
        if re.search(regex, texto, re.IGNORECASE):
            return empresa
    return None

In [8]:
def processar_relatorio(relatorio_path):
    texto_total = ""
    for imagem in os.listdir(relatorio_path):
        imagem_path = os.path.join(relatorio_path, imagem)
        if imagem.lower().endswith(('.png', '.jpg', '.jpeg', '.tiff', '.bmp')):
            try:
                # Extrair texto da imagem usando Tesseract
                texto = pytesseract.image_to_string(Image.open(imagem_path))
                texto_total += texto + "\n"
            except Exception as e:
                print(f"Erro ao processar {imagem_path}: {e}")
    return texto_total

In [ ]:
relatorios = [relatorio for relatorio in os.listdir(PROCESSED_PDF) if os.path.isdir(os.path.join(PROCESSED_PDF, relatorio))]
for relatorio in tqdm(relatorios, desc="Classificando relatórios"):
    relatorio_path = os.path.join(PROCESSED_PDF, relatorio)
    
    texto_relatorio = processar_relatorio(relatorio_path)
    empresa = identificar_empresa(texto_relatorio)
    
    if empresa:
        empresa_dir = os.path.join(ORGANIZED_PDF, empresa)
        os.makedirs(empresa_dir, exist_ok=True)

        shutil.move(relatorio_path, os.path.join(empresa_dir, relatorio))
    else:
        print(f"Empresa não identificada para: {relatorio}")

Classificando relatórios:  39%|███▉      | 19/49 [06:14<11:32, 23.07s/it]

Empresa não identificada para: audit_report_rtrs67


Classificando relatórios:  61%|██████    | 30/49 [10:02<07:05, 22.42s/it]

Empresa não identificada para: audit_report_rtrs68


Classificando relatórios:  78%|███████▊  | 38/49 [12:44<04:01, 22.00s/it]

...

Classificando relatórios:  90%|████████▉ | 44/49 [15:00<02:18, 27.75s/it]

Empresa não identificada para: audit_report_rtrs87


Classificando relatórios:  98%|█████████▊| 48/49 [16:48<00:29, 29.84s/it]

Empresa não identificada para: audit_report_rtrs85


Classificando relatórios: 100%|██████████| 49/49 [17:11<00:00, 21.06s/it]


### passo 3: conhecendo a distribuição das classes

In [9]:
#função para contar o número de subdiretórios (relatórios) por classe
def contar_relatorios(base_dir):
    classes = {}
    for empresa in os.listdir(base_dir):
        empresa_path = os.path.join(base_dir, empresa)
        if os.path.isdir(empresa_path):
            relatorios = [item for item in os.listdir(empresa_path) if os.path.isdir(os.path.join(empresa_path, item))]
            classes[empresa] = len(relatorios)
    return classes

#função para calcular o número mínimo de relatórios por classe
def estimar_relatorios_minimos(classes, min_relatorios=5):
    relatorios_insuficientes = {classe: count for classe, count in classes.items() if count < min_relatorios}
    return relatorios_insuficientes

In [ ]:
classes_contagem = contar_relatorios(ORGANIZED_PDF)
minimo_relatorios = 5  
relatorios_insuficientes = estimar_relatorios_minimos(classes_contagem, minimo_relatorios)

print("Contagem de relatórios por classe:")
for classe, count in classes_contagem.items():
    print(f"{classe}: {count} relatórios")

print("\nClasses com relatórios insuficientes:")
if relatorios_insuficientes:
    for classe, count in relatorios_insuficientes.items():
        print(f"{classe}: {count} (necessário: {minimo_relatorios})")
else:
    print("Todas as classes possuem relatórios suficientes.")

Contagem de relatórios por classe:
Control Union: 103 relatórios
SGS Argentina: 2 relatórios
Food Chain ID: 82 relatórios
Genesis Certificações: 44 relatórios
Bureau Veritas: 6 relatórios
Cert ID: 2 relatórios

Classes com relatórios insuficientes:
SGS Argentina: 2 (necessário: 5)
Cert ID: 2 (necessário: 5)


### passo 4: selecionando amostras e construir os arquivos json

In [10]:
import cv2

In [11]:
boxes = []
drawing = False
ix, iy = -1, -1
scale_percent = 50  #reduz a imagem para 50% do tamanho original -- obs: a tela do meu pc é pequena

# Função de callback para desenhar bounding boxes
def draw_bbox(event, x, y, flags, param):
    global ix, iy, drawing, boxes, img_resized, img_copy_resized

    if event == cv2.EVENT_LBUTTONDOWN:
        drawing = True
        ix, iy = x, y

    elif event == cv2.EVENT_MOUSEMOVE:
        if drawing:
            img_copy_resized = img_resized.copy()
            cv2.rectangle(img_copy_resized, (ix, iy), (x, y), (0, 255, 0), 2)

    elif event == cv2.EVENT_LBUTTONUP:
        drawing = False
        x_min, y_min = min(ix, x), min(iy, y)
        x_max, y_max = max(ix, x), max(iy, y)
        
        original_bbox = [
            int(x_min / scale_percent * 100),
            int(y_min / scale_percent * 100),
            int(x_max / scale_percent * 100),
            int(y_max / scale_percent * 100),
        ]
        boxes.append(original_bbox)
        cv2.rectangle(img_copy_resized, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)
        print(f"BBox salva: {original_bbox}")

# Função principal para processar uma imagem
def processar_imagem(image_path: Path, label: str):
    global img, img_resized, img_copy_resized, boxes

    boxes = []

    img = cv2.imread(str(image_path))
    if img is None:
        print(f"Erro ao carregar a imagem: {image_path}")
        return None

    #redimensiona a imagem para caber na tela
    width = int(img.shape[1] * scale_percent / 100)
    height = int(img.shape[0] * scale_percent / 100)
    img_resized = cv2.resize(img, (width, height), interpolation=cv2.INTER_AREA)
    img_copy_resized = img_resized.copy()

    cv2.namedWindow("image")
    cv2.setMouseCallback("image", draw_bbox)

    print("Pressione 'q' para sair.")
    while True:
        cv2.imshow("image", img_copy_resized)
        key = cv2.waitKey(1) & 0xFF
        if key == ord("q"):  #sair com 'q'
            break

    cv2.destroyAllWindows()

    return [{"bbox": bbox, "label": label} for bbox in boxes]

In [ ]:
image_path = ORGANIZED_PDF / 'Control Union' / 'audit_report_rtrs4' / 'page_1.jpg'  
label = "endereço" 
dados = processar_imagem(image_path, label)

qt.qpa.plugin: Could not find the Qt platform plugin "wayland" in "/home/cleyton/.local/lib/python3.10/site-packages/cv2/qt/plugins"


Pressione 'q' para sair.
BBox salva: [360, 686, 886, 735]


In [12]:
variaveis = [
    'endereço',
    'cod_certificado',
    'validade_certificado',
    'nome_organização',
    'tipo_avaliacao',
    'area_total',
    'producao',
    'data_prox_auditoria',
    'criterio_1_1',
    'criterio_1_2',
    'criterio_1_3',
    'criterio_2_1',
    'criterio_2_2',
    'criterio_2_3',
    'criterio_2_4',
    'criterio_2_5',
    'criterio_3_1',
    'criterio_3_2',
    'criterio_3_3',
    'criterio_3_4',
    'criterio_4_1',
    'criterio_4_2',
    'criterio_4_3',
    'criterio_4_4',
    'criterio_4_5',
    'criterio_5_1',
    'criterio_5_2',
    'criterio_5_3',
    'criterio_5_4',
    'criterio_5_5',
    'criterio_5_6',
    'criterio_5_7',
    'criterio_5_8',
    'criterio_5_9',
    'criterio_5_10',
    'criterio_5_11',
    'conformidade?',
]

In [ ]:
limite_classificacao = {
    "Control Union": 10,
    "SGS Argentina": 2,
    "Food Chain ID": 10,
    "Genesis Certificações": 3,
    "Bureau Veritas": 3,
    "Cert ID": 2,
}

In [14]:
import json
import time

def ordenar_paginas_numericamente(paginas):
    def extrair_numero(arquivo):
        match = re.search(r'(\d+)', arquivo.name)
        return int(match.group(1)) if match else float('inf')  
    return sorted(paginas, key=extrair_numero)


# Função que processa cada relatório individualmente
def processar_relatorio(relatorio_path, variaveis):
    resultados = {}
    paginas = ordenar_paginas_numericamente(relatorio_path.glob("*.jpg"))  # Ordenar numericamente

    for variavel in variaveis:
        print(f"Extraindo variável: {variavel}")
        pagina = input(f"Escolha a página (1-{len(paginas)}) para a variável '{variavel}': ")
        pagina_index = int(pagina) - 1

        if pagina_index < 0 or pagina_index >= len(paginas):
            print("Página inválida, pulando variável.")
            continue

        img_path = paginas[pagina_index]
        print(f"Processando imagem: {img_path}")

        dados = processar_imagem(img_path, variavel)
        if dados:
            resultados[variavel] = {"bbox": dados[0]["bbox"], "pagina": str(img_path.name)}
        else:
            print(f"Nenhum bounding box definido para '{variavel}', pulando.")

        time.sleep(2)

    return resultados


# Função que processa várias empresas e seus relatórios
def processar_empresas(base_dir, variaveis, limite_classificacao):
    """
    base_dir: Path para a pasta onde estão as subpastas de empresas.
    variaveis: lista de variáveis que você deseja extrair.
    limite_classificacao: dicionário { 'nome_da_empresa': quantidade_de_relatorios_a_processar }
    """
    for empresa, limite in limite_classificacao.items():
        empresa_dir = base_dir / empresa
        if not empresa_dir.exists():
            print(f"Diretório da empresa '{empresa}' não encontrado. Pulando.")
            continue

        relatorios = sorted(empresa_dir.iterdir())

        for i, relatorio in enumerate(relatorios[:limite]):
            output_path = relatorio / f"{relatorio.name}.json"

            if output_path.exists():
                print(f"Arquivo JSON já existe para '{relatorio.name}', pulando.")
                continue

            print(f"\n[Empresa: {empresa}] Processando relatório {i + 1}/{limite}: {relatorio.name}")
            resultados = processar_relatorio(relatorio, variaveis)

            output_path.parent.mkdir(parents=True, exist_ok=True)

            with open(output_path, "w", encoding="utf-8") as f:
                json.dump(resultados, f, indent=4, ensure_ascii=False)

            print(f"Resultados salvos em: {output_path}")


In [ ]:
processar_empresas(PROCESSED_PDF, variaveis, limite_classificacao)


[Empresa: Cert ID] Processando relatório 1/2: audit_report_rtrs242
Extraindo variável: endereço


ValueError: invalid literal for int() with base 10: ''

### passo 5: criando o dataset
- fazer o ocr de cada página com bbox
- alinhar os tokens retornados do ocr com os bbox do json
- gerar um dataset token-level (BIO tagging)

In [27]:
#funções auxiliares

def load_json(json_path):
    """Carrega um arquivo JSON e retorna o dicionário."""
    with open(json_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    return data

def compute_iou(bbox1, bbox2):
    """
    Calcula a IOU (Interseção sobre União) entre dois bounding boxes.
    bbox em formato [x1, y1, x2, y2].
    Retorna valor entre 0 e 1.
    """
    x1, y1, x2, y2 = bbox1
    x1b, y1b, x2b, y2b = bbox2

    inter_x1 = max(x1, x1b)
    inter_y1 = max(y1, y1b)
    inter_x2 = min(x2, x2b)
    inter_y2 = min(y2, y2b)

    if inter_x2 < inter_x1 or inter_y2 < inter_y1:
        return 0.0

    inter_area = (inter_x2 - inter_x1) * (inter_y2 - inter_y1)
    area1 = (x2 - x1) * (y2 - y1)
    area2 = (x2b - x1b) * (y2b - y1b)
    iou = inter_area / float(area1 + area2 - inter_area)
    return iou

def bbox_center_in(bbox_token, bbox_field):
    """
    Verifica se o centro do token está dentro do bbox_field.
    (Alternativa à IOU para decidir se o token está dentro do campo)
    """
    x1, y1, x2, y2 = bbox_token
    xf1, yf1, xf2, yf2 = bbox_field

    cx = (x1 + x2) / 2
    cy = (y1 + y2) / 2

    return (xf1 <= cx <= xf2) and (yf1 <= cy <= yf2)

def run_ocr_na_imagem(img_path):
    """
    Roda o Tesseract OCR na imagem, retornando uma lista de dicionários:
    [
      {
        "text": "Certificado",
        "bbox": [x1, y1, x2, y2],   # em pixels
      },
      ...
    ]
    As coordenadas vêm de (left, top, right, bottom).
    """
    image = Image.open(img_path)

    # Se necessário, aponte o caminho do executável do Tesseract:
    # pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

    # image_to_data retorna colunas: level, page_num, block_num, ...
    data = pytesseract.image_to_data(image, output_type=pytesseract.Output.DICT)

    tokens = []
    n_boxes = len(data['text'])
    for i in range(n_boxes):
        text = data['text'][i].strip()
        conf = int(data['conf'][i])
        if text == "" or conf < 0:
            continue

        x = data['left'][i]
        y = data['top'][i]
        w = data['width'][i]
        h = data['height'][i]
        x1, y1 = x, y
        x2, y2 = x + w, y + h

        tokens.append({
            "text": text,
            "bbox": [x1, y1, x2, y2]
        })
    return tokens

In [38]:
def build_dataset_from_reports(reports_dir, iou_threshold=0.5, use_center=False):
    """
    Percorre uma estrutura de dois níveis:
      1. Empresa (ex.: 'Control Union', 'Food Chain ID')
      2. Relatório (subpasta, ex.: 'Relatorio_A'), 
         onde existe 1 JSON e várias páginas .jpg.

    Para cada relatório, carrega o JSON (com as anotações de campos),
    roda OCR nas imagens e gera um arquivo .jsonl rotulado.
    """

    # 1) Percorrer cada pasta de EMPRESA
    for empresa_dir in Path(reports_dir).iterdir():
        if not empresa_dir.is_dir():
            continue  # ignora se não for diretório
        print(f"== Empresa: {empresa_dir.name}")

        # 2) Dentro de cada empresa, percorrer as subpastas de RELATÓRIOS
        for relatorio_dir in sorted(empresa_dir.iterdir()):
            if not relatorio_dir.is_dir():
                continue  # pula se não for diretório

            print(f"   >> Relatório: {relatorio_dir.name}")
            
            # Procurar arquivo JSON dentro do relatório
            json_files = list(relatorio_dir.glob("*.json"))
            if len(json_files) == 0:
                print(f"   [AVISO] Nenhum JSON encontrado em '{relatorio_dir.name}'. Pulando.")
                continue
            if len(json_files) > 1:
                print(f"   [AVISO] Mais de um JSON encontrado em '{relatorio_dir.name}'. Usando o primeiro.")
            json_file = json_files[0]

            # Carregar as anotações do JSON
            annotations = load_json(json_file)

            # Dicionário para armazenar tokens OCR por página
            ocr_results = {}  # { "page_1.jpg": [ {text, bbox, label}, ... ] }

            # Localizar as imagens (.jpg) do relatório
            image_files = sorted(relatorio_dir.glob("*.jpg"))
            if not image_files:
                print(f"   [AVISO] Não há arquivos .jpg em '{relatorio_dir.name}'. Pulando.")
                continue

            # 1) Rodar OCR em cada página e iniciar label como "O"
            for img_path in image_files:
                page_name = img_path.name  # ex.: "page_1.jpg"
                tokens_ocr = run_ocr_na_imagem(img_path)  # Função que faz OCR com pytesseract
                for t in tokens_ocr:
                    t["label"] = "O"
                ocr_results[page_name] = tokens_ocr

            # 2) Para cada campo no JSON, associar tokens do OCR
            for field_name, field_info in annotations.items():
                page_name = field_info.get("pagina")
                field_bbox = field_info.get("bbox")  # [x1, y1, x2, y2]

                if page_name not in ocr_results:
                    print(f"   [AVISO] '{page_name}' não encontrado em '{relatorio_dir.name}'. Pulando campo '{field_name}'.")
                    continue

                tokens_pagina = ocr_results[page_name]
                indices_tokens = []

                # Verificar cada token para ver se ele está dentro do bbox
                for idx, token in enumerate(tokens_pagina):
                    token_bbox = token["bbox"]
                    if use_center:
                        # se o centro do token cair dentro do bbox do campo
                        if bbox_center_in(token_bbox, field_bbox):
                            indices_tokens.append(idx)
                    else:
                        # se a IOU >= iou_threshold
                        iou_val = compute_iou(token_bbox, field_bbox)
                        if iou_val >= iou_threshold:
                            indices_tokens.append(idx)

                # Marcar tokens com B- e I-
                if len(indices_tokens) > 0:
                    indices_tokens.sort()
                    tokens_pagina[indices_tokens[0]]["label"] = f"B-{field_name}"
                    for i in indices_tokens[1:]:
                        tokens_pagina[i]["label"] = f"I-{field_name}"

            # 3) Gerar dataset final (token-level) para esse relatório
            output_path = relatorio_dir / f"{relatorio_dir.name}_labeled.jsonl"

            final_data = []
            for page_name, tokens_list in ocr_results.items():
                for token in tokens_list:
                    final_data.append({
                        "empresa": empresa_dir.name,
                        "relatorio": relatorio_dir.name,
                        "pagina": page_name,
                        "text": token["text"],
                        "bbox": token["bbox"],
                        "label": token["label"]
                    })

            # Salvar em JSON lines
            with open(output_path, "w", encoding="utf-8") as f_out:
                for item in final_data:
                    f_out.write(json.dumps(item, ensure_ascii=False) + "\n")

            print(f"   [INFO] Dataset anotado salvo em: {output_path}")

In [ ]:
build_dataset_from_reports(PROCESSED_PDF, iou_threshold=0.5, use_center=False)

== Empresa: Control Union
   >> Relatório: audit_report_rtrs101
   [INFO] Dataset anotado salvo em: /home/cleyton/Documentos/ProjetosGit/meu_repositorio_dissertacao/data/processed_data/organized_audit_reports/Control Union/audit_report_rtrs101/audit_report_rtrs101_labeled.jsonl
   >> Relatório: audit_report_rtrs104
   [AVISO] Nenhum JSON encontrado em 'audit_report_rtrs104'. Pulando.
   >> Relatório: audit_report_rtrs105
   [AVISO] Nenhum JSON encontrado em 'audit_report_rtrs105'. Pulando.
   >> Relatório: audit_report_rtrs110
   [AVISO] Nenhum JSON encontrado em 'audit_report_rtrs110'. Pulando.
   >> Relatório: audit_report_rtrs117
   [INFO] Dataset anotado salvo em: /home/cleyton/Documentos/ProjetosGit/meu_repositorio_dissertacao/data/processed_data/organized_audit_reports/Control Union/audit_report_rtrs117/audit_report_rtrs117_labeled.jsonl
   >> Relatório: audit_report_rtrs118
   [INFO] Dataset anotado salvo em: /home/cleyton/Documentos/ProjetosGit/meu_repositorio_dissertacao/data/

In [86]:
dataset_path = ORGANIZED_PDF / 'Control Union' / 'audit_report_rtrs101' / 'audit_report_rtrs101_labeled.jsonl'

with open(dataset_path, "r", encoding="utf-8") as f:
    for line in f:
        record = json.loads(line.strip())
        print(record)

{'empresa': 'Control Union', 'relatorio': 'audit_report_rtrs101', 'pagina': 'page_1.jpg', 'text': '-', 'bbox': [186, 126, 283, 219], 'label': 'O'}
{'empresa': 'Control Union', 'relatorio': 'audit_report_rtrs101', 'pagina': 'page_1.jpg', 'text': 'CONTROLUNION', 'bbox': [313, 122, 782, 202], 'label': 'O'}
{'empresa': 'Control Union', 'relatorio': 'audit_report_rtrs101', 'pagina': 'page_1.jpg', 'text': 'Relatério', 'bbox': [293, 283, 418, 307], 'label': 'O'}
{'empresa': 'Control Union', 'relatorio': 'audit_report_rtrs101', 'pagina': 'page_1.jpg', 'text': 'de', 'bbox': [428, 283, 460, 307], 'label': 'O'}
{'empresa': 'Control Union', 'relatorio': 'audit_report_rtrs101', 'pagina': 'page_1.jpg', 'text': 'Resumo', 'bbox': [471, 285, 580, 307], 'label': 'O'}
{'empresa': 'Control Union', 'relatorio': 'audit_report_rtrs101', 'pagina': 'page_1.jpg', 'text': 'Publico', 'bbox': [591, 283, 690, 307], 'label': 'O'}
{'empresa': 'Control Union', 'relatorio': 'audit_report_rtrs101', 'pagina': 'page_1.jpg